In [90]:
# 나의 구글 드라이브를 구글 본사 서버와 연결하는 작업
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [91]:
# 현재 실행 경로 확인하기
import os
print(os.getcwd())

/content/drive/My Drive/nlp


In [92]:
# 현재 작업 폴더 nlp로 경로 이동하기
%cd '/content/drive/My Drive/nlp'

/content/drive/My Drive/nlp


In [93]:
print(os.getcwd())

/content/drive/My Drive/nlp


In [94]:
# 필요한 라이브러리 임폴트
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [95]:
"""
KoNLpy(코엔엘파이) 라이브러리 설치
KoNLpy(코엔엘파이) 라이브러리 설치 : 한국어 형태소 분석기로서, 한글 자연어 처리를 위해
만들어진 라이브러리
"""

# KoNLpy 라이브러리 설치
!pip install konlpy

In [96]:
# 필요한 라이브러리 임폴트
import konlpy
from konlpy.tag import Kkma, Okt
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [97]:
# 한글 형태소 분석기 생성

kkma = Kkma()
okt = Okt()

In [98]:
# pandas.read_csv() 함수를 사용하여 데이터프레임 생성하기
df = pd.read_csv('3rd_last_project.csv')
#pd.options.display.max_columns = None
print("df.title = \n", df.title)
print("df.content = \n", df.content)

df.title = 
 0      광주광역시 청년 지원사업의 모든것!! 2021년 청년정책 종합안내\t\t\t\t\t...
1                            2021년 청년 13(일+삶통장 6기 참여자 모집
2      2021년 청년13(일+삶통장 지원사업 선정자 명단 및 예비후보자 명단공고\t\t\...
3                  고졸청년 특화 헬스케어 분야 디지털 마케팅일자리, 창출 훈련생 모집
4                                  2021 행복플러스 건강지원 사업 안내
                             ...                        
117                                           철학으로 읽는 옛집
118             코로나 극복 예술프로젝트 ‘슬기로운 문화생활, GAC유튜브 안방예술극장’
119                         코로나19 예방접종 이상반응 피해보상 구비서류 안내
120    필리핀 전래동화 ‘바닷물은 왜 짤까?’를 바탕으로 인간과 자연의 평화로운 공존을 이...
121    이야기꽃도서관 독서문화프로그램 및 그림책 활동가 동아리 프로그램 등 기타 행사 운영...
Name: title, Length: 122, dtype: object
df.content = 
 0      {기간:2021-04-27 ~2021-09-30}\n{대상:1991.01.01~20...
1      {기간:2021-04-27 ~2021-10-20}\n{대상:1991.01.01~20...
2      {기간:2021-04-27 ~2021-10-20}\n{대상:1991.01.01~20...
3      {기간:2021-06-25 ~2021-12-13}\n{대상:1991.01.01~20...
4      {기간:2021-06-11 ~2021-12-13}\n{대상:1991.01.01~20...
                    

In [99]:
#결측치 확인
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    122 non-null    object
 1   content  122 non-null    object
 2   url      122 non-null    object
dtypes: object(3)
memory usage: 3.0+ KB
None


In [100]:
print(list(df.title))
df_tmp1 = pd.DataFrame(list(df.title), columns=['title'])
print(df_tmp1)

['광주광역시 청년 지원사업의 모든것!! 2021년 청년정책 종합안내\t\t\t\t\t\t\t\t', '2021년 청년 13(일+삶통장 6기 참여자 모집', '2021년 청년13(일+삶통장 지원사업 선정자 명단 및 예비후보자 명단공고\t\t\t\t\t', '고졸청년 특화 헬스케어 분야 디지털 마케팅일자리, 창출 훈련생 모집', '2021 행복플러스 건강지원 사업 안내', '마음건강상담소 오픈-광주청년센터\t', '청년의 일터를 지키는 N개의 청년일기 사연모집', '광주청년드림은행 6월 가계부 워크샵 안내\t', "2021년 '청년교통수당 드림' 참여자 모집공고\t\t\t\t\t\t\t\t", '광주청년 일경험드립 사업 제10기 드림청년(참여청년 공개모집 연장공고', '의료기관아동학대, 긴급복지 신고의무자 교육 및 범죄 조회 공지', '광주글로벌모터스 AI역량검사 지원부스 운영', '광주글로벌모터스 채용설명 취업특강<7.23(금, 7.26(월> 운영 안', '[2021 국민참여 청렴콘텐츠 공모전] 2차 소문내기 이벤트 시작', '[2021 국민참여 청렴콘텐츠 공모전] 소문내기 이벤트', '사회적거리두기 2단계 격상에 따른 광산구립도서관 운영 안내', '2020년도 김대중컨벤션센터 전시장 정시모집 종료 및 수시대관 실시 안내', '[공지]2021년 12월 다목적홀 대관 공고', '[공지]2021년 광주광역시 우수전시회 지원사업 모집공고', '[공지]2021년 광주광역시 우수전시회 지원사업 재 모집 공고', '[공지]2021년도 김대중컨벤션센터 전시장 정시대관 종료 및 수시대관 실시 안내', '[공지]2022년 상반기 전시장 정시모집 공고', '[공지]2022년 상반기 전시장 정시모집 재 공고', '[공지]김대중컨벤션센터 대표홈페이지 (장기 미접속자 개인정보 파기 안내', '[공지]김대중컨벤션센터 상설전시장 및 회의실 로비 임대요율 변경 안내', '[공지]김대중컨벤션센터 임대요율 인상 안내', '[공지]김대중컨벤션센터 전시장 임대료 월별 차등율 변경 안내', '[공

In [101]:
df_tmp = pd.DataFrame([df.title, df.content],columns=['title','content'])
print(df_tmp.head())

         title  content
title      NaN      NaN
content    NaN      NaN


In [102]:
#데이터 (타이틀) 전처리 
import re
sentences = [data for data in df.title]

for data in sentences:

  #print("data = ", data)
  re.sub('[^0-9a-zA-Zㄱ-힗]+', '', data)
  # cleaned_text.append(data)


text = sentences

stop_words = ['■','】','\n','\\n','\\n','\\\n','\n\n','{', '}', '[', ']', '\’', '?', '!', 'ㄴ', 'ㄹ','\"', ')', '(', '...', '\'',',', '.']
re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', data)

# print("타이틀:",text)
print("타이틀갯수:",len(text))

타이틀갯수: 122


In [103]:
#문장을 토큰화 해서 사전 만들기

text_title = df.title
text_content = df.content


#토큰화
sentences_kor = word_tokenize(str(text_title))
sentences_kor2 = word_tokenize(str(text_content))

#결과확인
# print(sentences_kor)
# print(sentences_kor2)

In [104]:
# 필요한 라이브러리 임폴트
import konlpy
from konlpy.tag import Kkma, Okt

In [105]:
# 한글 형태소 분석기 생성

kkma = Kkma()
okt = Okt()

In [106]:

text_title2=str(text_title)
text_title2.split()
text_content2=str(text_content)
text_content2.split()

# kkma 이용

morphs_kkma = kkma.morphs(text_title2)
morphs_kkma2 = kkma.morphs(text_content2)
# print("morphs_kkma의 결과 = \n ", morphs_kkma)
# print("morphs_kkma2의 결과 = \n ", morphs_kkma2)

# #Okt 이용
# morphs_okt = okt.morphs(text_title2)
# morphs_okt2 = okt.morphs(text_content2)
# print("morghs_okt의 결과 = \n", morphs_okt)
# print("morghs_okt2의 결과 = \n", morphs_okt2)

#비교 : 공백 단위로 분할
result = text_title2.split()
result2 = text_content2.split()
# print("공백 단위로 분할한 결과= \n", result)
# print("공백 단위로 분할한 결과2= \n", result2)

In [107]:
### 형태소와 품사를 동시에 추출 --> pos() 함수 이용

#kkma 이용
pos_kkma = kkma.pos(text_title2)
pos_kkma2 = kkma.pos(text_content2)
# print("pos_kkma = \n",pos_kkma)
# print("pos_kkma2 = \n",pos_kkma)

#Okt 이용
pos_okt = okt.pos(text_title2)
pos_okt2 = okt.pos(text_content2)
# print("pos_okt = \n",pos_okt)
# print("pos_okt2 = \n",pos_okt2)

In [108]:
###문장에서 명사만 추출

#kkma 이용
nouns_kkma = kkma.nouns(text_title2)
nouns_kkma2 = kkma.nouns(text_content2)
# print("nouns_kkma = \n", nouns_kkma)
# print("nouns_kkma2 = \n", nouns_kkma2)


In [109]:
# fit() 함수를 이용 --> 자동으로 단어 사전 생성
tfidf.fit(nouns_kkma2)

#생성된 단어 사전 확인하기 --> 단어 사전 : '단어의 수치화' 완성
vocab = tfidf.vocabulary_
# print(vocab)

NameError: ignored

In [110]:
#kkma로 추출된 명사로 기간과 대상 추출하기 위한 사전만들기
#그외 나머지는 내용 싹 다 담아라
test_date = ['기간', '접수 기간','기간 :']

test_people = ['대상', '대상자', '대상 :', '대상자 :']

In [111]:
# #기간 대상 추출하기_test
base_data = [data for data in df.content]
#print("base_data = \n", base_data[0])
print('\n')

# #stop_words = ['■','】','\n','\\n','\\n','\\\n','\n\n','{', '}', '[', ']', '\’', '?', '!', 'ㄴ', 'ㄹ','\"', ')', '(', '...', '\'',',', '.']
sentences = []
for data in base_data:    
  sentence_str = str(data)
  sentence = sentence_str.split('\n')
  sentences.append(sentence)
# print("sentences[0] = \n", sentences[0])

# # 기간
period_front = []
period_end = []
# # 대상
run = []


for sentence in sentences:
  for data in sentence:
    print("data = ", data)
    if '{기간:}' in data[:7] or '{기간}' in data[:7] or '기간:' in data[:7]:
      period_front.append(data[4:14])
      period_end.append(data[16:-1])



for sentence in sentences:
  for data in sentence:
    print("data = ", data)
    if '{대상:}' in data[:7] or '{대상}' in data[:7] or '대상:' in data[:7]:
      run.append(data[4:-1])

# print(period[0])
# print(len(period[0]))

# 결과 확인
# print("기간-front", period_front)
# print("기간-end", period_end)
# print("대상", run)

# cleaned_period  = []
# for data in period:
#   if ':' in data:
#     cleaned_period.append(data)
#   else:
#     continue

# print(cleaned_period)


zeros = [0] * (122 - len(run))

result_period = run  + zeros


# print(result_period)
# print(f"기간 결과의 길이 = {len(result_period)}")



data =  {기간:2021-04-27 ~2021-09-30}
data =  {대상:1991.01.01~2002.01.01}
data =  2021년 청년13(일+삶통장 지원사업 선정자 명단 및 예비후보자 명단공고,작성자 : 청년금융복지지원센터,등록일 : 2021-04-27 08:40:13,조회수 : 1756,첨부파일 1,2021년_청년13통장_6기_선정자_공고문.hwp [13 KB],첨부파일 2,2021년_청년13통장_6기_선정자_및_예비자_명단.xlsx [37 KB],, 일하는 청년의 더 나은 삶을 위해 임금소득이 낮은 근로청년의 소액 단기 저축을 지원하는,‘2021년 청년 13(일+삶통장 지원 사업’ 선정자 명단 및 예비후보자 명단을 공고합니다,광 주 광 역 시 장 ,목록,다음글,610. [안내]마음건강상담소 오픈-광주청년센터 청년금융복지지원센터,이전글,581. [공지] 광주광역시 청년 지원사업의 모든것!! 2021년 청년정책 종합안내,청년금융복지지원센터
data =  {기간:2021-04-27 ~2021-10-20}
data =  {대상:1991.01.01~2002.12.31}
data =  [공지] 광주광역시 청년 지원사업의 모든것!! 2021년 청년정책 종합안내,작성자 : 청년금융복지지원센터,등록일 : 2021-04-27 08:40:46,조회수 : 2836,첨부파일 1,young2021_file01.pdf [2838 KB],    2021년 청년 정책종합 안내,21년 광주광역시 청년들을 위한 지원사업에 모든것!!!,첨부파일 참조, ,  목록,다음글,582. [공지]2021년 청년13(일+삶통장 지원사업 선정자 명단 및 예비후보자 명단공고 청년금융복지지원센터,이전글,544. [공지]2021년 청년 13(일+삶통장 6기 참여자 모집!!!!,청년금융복지지원센터
data =  {기간:2021-04-27 ~2021-10-20}
data =  {대상:1991.01.01~2002.12.31}
data =  [안내]마음건강상담소 오픈-광주청년센터,작성자 : 

In [112]:
(len(run))

122

In [113]:
# pandas.read_csv() 함수를 사용하여 데이터프레임 생성하기
# df = pd.read_csv('.csv')
df2 = pd.read_csv('3rd_last_project_2.csv')

pd.options.display.max_columns = None

dict_ = {'title':list(df2.title), 'content':list(df2.content),'end_date':period_end,'target':run, 'url':list(df2.url)}

# 기간(start, end), 대상, 타이틀 4가지 데이터를 하나의 데이터프레임으로 만들기
import pandas as pd
df_cleaned = pd.DataFrame(dict_)
# print("결과 확인", df_cleaned.info())
# print(df_cleaned.head())

# # 결과를 pickle로 저장하기
# df_cleaned.to_pickle('df_cleaned.pkl')

df_cleaned.to_csv('last_dic.csv' ,encoding ='utf-8')

In [ ]:
# pickle 파일 형태로 저장되어 있는 df_cleaned.pkl 파일을 불러와서 읽기
# pd.options.display.max_columns = None
# df_cleaned = pd.read_pickle('df_cleaned.csv')
# print("결과확인", df_cleaned)

# df_cleaned.to_csv('last_dic.csv' ,encoding ='utf-8')

NameError: ignored

In [114]:
df_cleaned

,title,content,end_date,target,url
0,광주광역시 청년 지원사업의 모든것!! 2021년 청년정책 종합안내\t\t\t\t\t...,"2021년 청년13(일+삶통장 지원사업 선정자 명단 및 예비후보자 명단공고,작성자 ...",2021-09-30,1991.01.01~2002.01.01,https://www.13account.or.kr/board/view/notice/581
1,2021년 청년 13(일+삶통장 6기 참여자 모집,"[공지] 광주광역시 청년 지원사업의 모든것!! 2021년 청년정책 종합안내,작성자 ...",2021-10-20,1991.01.01~2002.12.31,https://www.13account.or.kr/board/view/notice/582
2,2021년 청년13(일+삶통장 지원사업 선정자 명단 및 예비후보자 명단공고\t\t\...,"[안내]마음건강상담소 오픈-광주청년센터,작성자 : 청년금융복지지원센터,등록일 : 2...",2021-10-20,1991.01.01~2002.12.31,https://www.13account.or.kr/board/view/notice/581
3,"고졸청년 특화 헬스케어 분야 디지털 마케팅일자리, 창출 훈련생 모집","643. [청년지원사업안내]2021년 '청년교통수당 드림' 참여자 모집공고,작성자 ...",2021-12-13,1991.01.01~2002.12.31,https://www.13account.or.kr/board/view/notice/610
4,2021 행복플러스 건강지원 사업 안내,"638. [참고]광주청년드림은행 6월 가계부 워크샵 안내,작성자 : 청년금융복지지원...",2021-12-13,1991.01.01~2002.12.31,https://www.13account.or.kr/board/view/notice/640
...,...,...,...,...,...
117,철학으로 읽는 옛집,"토요인문학> 철학으로 읽는 옛집 기간(8/28, 9/25 ,•우리 옛집의 철학 ,•...",2021-09-25,1900.01.01~2021.01.01,https://www.seogu.gwangju.kr/library/index.9is...
118,"코로나 극복 예술프로젝트 ‘슬기로운 문화생활, GAC유튜브 안방예술극장’","광주문화예술회관에서 제안하는 슬기로운 문화생활,코로나 극복 예술프로젝트‘힘내라 대한...",2021-12-31,1900.01.01~2021.01.01,http://www.gjcf.or.kr/cf/Board/14616/detailVie...
119,코로나19 예방접종 이상반응 피해보상 구비서류 안내,"코로나19 예방접종 이상반응 피해보상 구비서류 안내,,> 신청대상 : 코로나19 예...",2021-12-31,1900.01.01~2021.01.01,https://health.gwangsan.go.kr/bb/bbBoard.php?a...
120,필리핀 전래동화 ‘바닷물은 왜 짤까?’를 바탕으로 인간과 자연의 평화로운 공존을 이...,"기간,2021.9.4.(토 - 9.5.(일,시간,11:00, 15:00,장소,어린이...",2021-09-05,1990.01.01~2021.01.01,https://www.acc.go.kr/main/performance.do?PID=...


In [115]:
doc =list(df2.content)

In [116]:
#필요한 라이브러리 임폴트
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer() 

In [117]:
# for data in doc:
#   data.fit_transform()

# vect 단어를 수치화(빈도수 기반), 문장을 백터화
result = vect.fit_transform(doc).toarray() 

In [118]:
# 생성된 단어 사전 확인
print(vect.vocabulary_)

{'2021년': 188, '청년13': 3990, '삶통장': 2283, '지원사업': 3811, '선정자': 2379, '명단': 1785, '예비후보자': 2906, '명단공고': 1786, '작성자': 3422, '청년금융복지지원센터': 3993, '등록일': 1678, '2021': 185, '04': 23, '27': 244, '08': 36, '40': 346, '13': 77, '조회수': 3694, '1756': 114, '첨부파일': 3983, '2021년_청년13통장_6기_선정자_공고문': 189, 'hwp': 632, 'kb': 648, '2021년_청년13통장_6기_선정자_및_예비자_명단': 190, 'xlsx': 759, '37': 313, '일하는': 3312, '청년의': 3997, '나은': 1418, '삶을': 2281, '위해': 3076, '임금소득이': 3322, '낮은': 1437, '근로청년의': 1280, '소액': 2430, '단기': 1517, '저축을': 3478, '지원하는': 3823, '청년': 3989, '지원': 3805, '사업': 2209, '명단을': 1788, '공고합니다': 1027, '목록': 1828, '다음글': 1507, '610': 431, '안내': 2693, '마음건강상담소': 1734, '오픈': 2948, '광주청년센터': 1182, '이전글': 3212, '581': 407, '공지': 1067, '광주광역시': 1129, '지원사업의': 3813, '모든것': 1798, '청년정책': 3999, '종합안내': 3707, '46': 361, '2836': 251, 'young2021_file01': 760, 'pdf': 715, '2838': 252, '정책종합': 3608, '21년': 211, '청년들을': 3995, '위한': 3074, '지원사업에': 3812, '참조': 3936, '582': 408, '544': 401, '6기': 454, '참여자': 3925, '

In [119]:
# 백터화된 문장을 확인
print(result)
print(f"백터로 처리된 문장의 수 = {len(result)}")
print(f"백터로 처리된 첫번째 문장 확인 = {result[0]}")

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [2 0 0 ... 0 0 0]
 [5 0 0 ... 0 0 0]]
백터로 처리된 문장의 수 = 122
백터로 처리된 첫번째 문장 확인 = [0 0 0 ... 0 0 0]


In [120]:
result

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 0, 0],
       [5, 0, 0, ..., 0, 0, 0]])

In [121]:
#코사인 유사도 비교하기 위한 함수 생성
#협업필터링
#유사도 추천서비스
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

In [122]:
df.content[10]

'{기간:2021-06-30 ~ 2021-12-31}\n{대상:1900.01.01~2021.01.01}\n의료기관 참고하시기 바랍니다.,제출방법 : 팩스(960-3737 또는 메일(gamnamu 골뱅이 korea.kr,,,1. 의료기관 아동학대 신고의무자 교육-의료인 대상,파일1 결과보고서,파일2 관련 안내문,,2. 의료기관 아동범죄, 성범죄, 노인학대범죄 취업조회-전체 종사자 대상(6.30.까지,파일3 인적사항회신서(가능하면 파일작성하시어 메일 송부 부탁드립니다,,3. 의료기관 긴급복지 신고의무자 교육-전체 종사자 대상,파일4 안내문 및 결과보고서(서식1 또는 서식3 작성하여 제출,파일5 관련 공문'

In [123]:
df2.title[10]

'의료기관아동학대, 긴급복지 신고의무자 교육 및 범죄 조회 공지'

In [124]:
result[10]

array([0, 0, 0, ..., 0, 0, 0])

In [125]:
result.shape

(122, 4606)

In [129]:
# for i in df.content:

  # print(cos_sim(df2.title[10],df2.title[i])) #콘텐츠10과 모든콘텐츠의 코사인 유사도
dic={}
for cur in range(result.shape[0]):
  dic[cur]={}
  for i in range(result.shape[0]):
    print(cos_sim(result[cur],result[i])) #콘텐츠10과 모든콘텐츠의 코사인 유사도
    dic[cur][i] = cos_sim(result[cur],result[i])


# dic


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
0.08251369970070348
0.13228756555322954
0.052223296786709346
0.08810677178572227
0.09901475429766741
0.13590417710722233
0.17411430002640282
0.16140042236349222
0.09231326627541017
0.14419211804559506
0.016666666666666666
0.04242640687119285
0.06123724356957946
0.04125684985035174
0.019802950859533482
0.16329931618554522
0.15228622596829317
0.1705924863737721
0.021821789023599235
0.08320502943378437
0.11666666666666667
0.09237604307034013
0.029070094986690553
0.017609018126512475
0.051987524491003634
0.04375949744936836
0.010943513103291655
0.0
0.10190493307301361
0.0
0.05827715174143585
0.0
0.0
0.08427009716003844
0.0753778361444409
0.10606601717798213
0.0659380473395787
0.04351426345763369
0.07179581586177382
0.041602514716892185
0.10454167469786334
0.15577356272359535
0.04601789933084223
0.03643900149394216
0.058834840541455206
0.09660917830792959
0.11684042705013718
0.11473525658871213
0.07495316889958614
0.11748539016153646
0.03076923076923077
0

In [127]:
df = pd.DataFrame(dic)

In [128]:
df[1].sort_values()[:-1]
# series[-8:].index
# series[:2].index

34    0.000000
59    0.000000
93    0.000000
31    0.000000
70    0.000000
        ...   
4     0.310853
3     0.330719
9     0.509919
2     0.559914
0     0.706686
Name: 1, Length: 121, dtype: float64